In [1]:
import os    as os

import numpy as np

import pandas as pd


In [2]:
working_directory  = "./"

loca2_inv_file     = "./LOCA2_inventory_min_errors_short.csv"

globus_root_dir    = "/data/GLOBUS/wjc"

thredds_root_dir   = "/projects/ECEP/LOCA_MACA_Ensembles/LOCA2/LOCA2_CONUS"

variables          = ["tasmax", 
                      "tasmin", 
                      "pr"]

scenarios          = ["historical", 
                      "ssp245", 
                      "ssp370", 
                      "ssp585"]

hist_year_strings  = ["1950-2014"] 

futr_year_strings  = ["2015-2044",
                      "2045-2074",
                      "2075-2100"]

version_strings    = ["LOCA_16thdeg_v20220413", # tasmax
                      "LOCA_16thdeg_v20220413", # tasmin
                      "LOCA_16thdeg_v20220519"] # pr

#subset_lon = np.array([-114.28125, -86.21875]) + 360
#subset_lat = np.array([  33.96875,  52.78125])

#subset_i   = [165,614]
#subset_j   = [141,442]


#bounds_script_command = 'defdim("bnds",2);time@bounds="time_bnds";time_bnds[$time,$bnds]=0.0;*time_dff=0.5*(time(1)-time(0));time_bnds(:,0)=time-time_dff;time_bnds(:,1)=time+time_dff;lat@bounds="lat_bnds";lat_bnds[$lat,$bnds]=0.0;*lat_dff=0.5*(lat(1)-lat(0));lat_bnds(:,0)=lat-lat_dff;lat_bnds(:,1)=lat+lat_dff;lon@bounds="lat_bnds";lon_bnds[$lon,$bnds]=0.0;*lon_dff=0.5*(lon(1)-lon(0));lon_bnds(:,0)=lon-lon_dff;lon_bnds(:,1)=lon+lon_dff;'


In [3]:
loca2_ensembles_list = pd.read_csv(filepath_or_buffer = loca2_inv_file)

models  = loca2_ensembles_list[ "Model"].values
members = loca2_ensembles_list["Member"].values

models  = models[0:1]
members = members[0:1]

print(models)
print(members)

['GFDL-ESM4']
['r1i1p1f1']


## Level 1 Workflow

Goal: Reduce File Size, Organize Metadata.

1) Create File and Directory Information to Arrive at File Processing Step
2) Purge Global Metadata (Will be rewritten neatly at a second date) and Variable History
```
   print('nohup ncatted -h -O -a  ,global,d,, ' + globus_file_loc)
```
3) Fix Lat and Lon Metadata to Match LOCA1 set.
```
   print('nohup ncatted -h -O -a              ,lat,d,,                 ' + globus_file_loc)
   print('nohup ncatted -h -O -a   description,lat,c,c,"Latitude"      ' + globus_file_loc)
   print('nohup ncatted -h -O -a     long_name,lat,c,c,"Latitude"      ' + globus_file_loc)
   print('nohup ncatted -h -O -a standard_name,lat,c,c,"latitude"      ' + globus_file_loc)
   print('nohup ncatted -h -O -a         units,lat,c,c,"degrees_north" ' + globus_file_loc)
   print('nohup ncatted -h -O -a          axis,lat,c,c,"Y"             ' + globus_file_loc)

   print('nohup ncatted -h -O -a              ,lon,d,,                ' + globus_file_loc)
   print('nohup ncatted -h -O -a   description,lon,c,c,"Longitude"    ' + globus_file_loc)
   print('nohup ncatted -h -O -a     long_name,lon,c,c,"Longitude"    ' + globus_file_loc)
   print('nohup ncatted -h -O -a standard_name,lon,c,c,"longitude"    ' + globus_file_loc)
   print('nohup ncatted -h -O -a         units,lon,c,c,"degrees_east" ' + globus_file_loc)
   print('nohup ncatted -h -O -a          axis,lon,c,c,"X"            ' + globus_file_loc)
```
4) Convert Units to LOCA1 Units (degC for temps, kg m-2 for pr) and compress using same math as LOCA1.
   *  tasmax 
```
   os.system("nohup ncap2 -h -d 8 --script 'where(tasmax > 1e9) tasmax=(-32493.85)'  " + globus_file_loc + ' ' + globus_file_loc + '.temp.nc' )
   print('nohup mv -fv ' + globus_file_loc + '.temp.nc' + ' ' +  globus_file_loc   )
        
   os.system("nohup ncap2 -h -d 8 --script 'tasmax=short(round((tasmax-273.15)*10))' " + globus_file_loc + ' ' + globus_file_loc + '.temp.nc' )
   print('nohup mv -fv ' + globus_file_loc + '.temp.nc' + ' ' +  globus_file_loc   )

   print('nohup ncatted -h -O -a              ,tasmax,d,,                                 ' + globus_file_loc)
   print('nohup ncatted -h -O -a   description,tasmax,c,c,"2-m Max Daily Air Temperature" ' + globus_file_loc)
   print('nohup ncatted -h -O -a     long_name,tasmax,c,c,"2-m Max Daily Air Temperature" ' + globus_file_loc)
   print('nohup ncatted -h -O -a standard_name,tasmax,c,c,"air_temperature"               ' + globus_file_loc)
   print('nohup ncatted -h -O -a         units,tasmax,c,c,"degC"                          ' + globus_file_loc)
   print('nohup ncatted -h -O -a  cell_methods,tasmax,c,c,"time: maximum"                 ' + globus_file_loc)
   print('nohup ncatted -h -O -a    _FillValue,tasmax,c,s,-32767                          ' + globus_file_loc)
   print('nohup ncatted -h -O -a    add_offset,tasmax,c,f,0                               ' + globus_file_loc)
   print('nohup ncatted -h -O -a  scale_factor,tasmax,c,f,0.1                             ' + globus_file_loc)
```
   *  tasmin 
```
   os.system("nohup ncap2 -h -d 8 --script 'where(tasmin > 1e9) tasmin=(-3003.55)'  " + globus_file_loc + ' ' + globus_file_loc + '.temp.nc' )
   print('nohup mv -fv ' + globus_file_loc + '.temp.nc' + ' ' +  globus_file_loc   )
        
   os.system("nohup ncap2 -h -d 8 --script 'tasmin=short(round((tasmin-273.15)*10))' " + globus_file_loc + ' ' + globus_file_loc + '.temp.nc' )
   print('nohup mv -fv ' + globus_file_loc + '.temp.nc' + ' ' +  globus_file_loc   )

   print('nohup ncatted -h -O -a              ,tasmin,d,,                                 ' + globus_file_loc)
   print('nohup ncatted -h -O -a   description,tasmin,c,c,"2-m Min Daily Air Temperature" ' + globus_file_loc)
   print('nohup ncatted -h -O -a     long_name,tasmin,c,c,"2-m Min Daily Air Temperature" ' + globus_file_loc)
   print('nohup ncatted -h -O -a standard_name,tasmin,c,c,"air_temperature"               ' + globus_file_loc)
   print('nohup ncatted -h -O -a         units,tasmin,c,c,"degC"                          ' + globus_file_loc)
   print('nohup ncatted -h -O -a  cell_methods,tasmin,c,c,"time: minimum"                 ' + globus_file_loc)
   print('nohup ncatted -h -O -a    _FillValue,tasmin,c,s,-32767                          ' + globus_file_loc)
   print('nohup ncatted -h -O -a    add_offset,tasmin,c,f,0                               ' + globus_file_loc)
   print('nohup ncatted -h -O -a  scale_factor,tasmin,c,f,0.1                             ' + globus_file_loc)
```
   *  pr (kg m-2 s-1)
```
   os.system("nohup ncap2 -h -d 8 --script 'where(pr > 1e9) pr=(-32767./864000.)' " + globus_file_loc + ' ' + globus_file_loc + '.temp.nc' )
   print('nohup mv -fv ' + globus_file_loc + '.temp.nc' + ' ' +  globus_file_loc   )
        
   os.system("nohup ncap2 -h -d 8 --script 'pr=short(round(pr*864000.))' " + globus_file_loc + ' ' + globus_file_loc + '.temp.nc' )
   print('nohup mv -fv ' + globus_file_loc + '.temp.nc' + ' ' +  globus_file_loc   )

   print('nohup ncatted -h -O -a              ,pr,d,,                             ' + globus_file_loc)
   print('nohup ncatted -h -O -a   description,pr,c,c,"Daily Total Precipitation" ' + globus_file_loc)
   print('nohup ncatted -h -O -a     long_name,pr,c,c,"Daily Total Precipitation" ' + globus_file_loc)
   print('nohup ncatted -h -O -a standard_name,pr,c,c,"precipitation_amount"      ' + globus_file_loc)
   print('nohup ncatted -h -O -a         units,pr,c,c,"kg m-2"                    ' + globus_file_loc)
   print('nohup ncatted -h -O -a  cell_methods,pr,c,c,"time: sum"                 ' + globus_file_loc)
   print('nohup ncatted -h -O -a    _FillValue,pr,c,s,-32767                      ' + globus_file_loc)
   print('nohup ncatted -h -O -a    add_offset,pr,c,f,0                           ' + globus_file_loc)
   print('nohup ncatted -h -O -a  scale_factor,pr,c,f,0.1                   ' + globus_file_loc)
```
   *  pr (kg m-2)
```
   os.system("nohup ncap2 -h -d 8 --script 'where(pr > 1e9) pr=(-3276.7)' " + globus_file_loc + ' ' + globus_file_loc + '.temp.nc' )
   print('nohup mv -fv ' + globus_file_loc + '.temp.nc' + ' ' +  globus_file_loc   )
        
   os.system("nohup ncap2 -h -d 8 --script 'pr=short(round(pr*10.))' " + globus_file_loc + ' ' + globus_file_loc + '.temp.nc' )
   print('nohup mv -fv ' + globus_file_loc + '.temp.nc' + ' ' +  globus_file_loc   )

   print('nohup ncatted -h -O -a              ,pr,d,,                             ' + globus_file_loc)
   print('nohup ncatted -h -O -a   description,pr,c,c,"Daily Total Precipitation" ' + globus_file_loc)
   print('nohup ncatted -h -O -a     long_name,pr,c,c,"Daily Total Precipitation" ' + globus_file_loc)
   print('nohup ncatted -h -O -a standard_name,pr,c,c,"precipitation_amount"      ' + globus_file_loc)
   print('nohup ncatted -h -O -a         units,pr,c,c,"kg m-2"                    ' + globus_file_loc)
   print('nohup ncatted -h -O -a  cell_methods,pr,c,c,"time: sum"                 ' + globus_file_loc)
   print('nohup ncatted -h -O -a    _FillValue,pr,c,s,-32767                      ' + globus_file_loc)
   print('nohup ncatted -h -O -a    add_offset,pr,c,f,0                           ' + globus_file_loc)
   print('nohup ncatted -h -O -a  scale_factor,pr,c,f,0.1                   ' + globus_file_loc)
```


In [4]:
for m in range(len(models)):
    
    model  =  models[m]
    member = members[m]
    
    for scenario in scenarios:
        
        print("# XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX")
      
        if (scenario == "historical"):
            year_strings = hist_year_strings
        else:
            year_strings = futr_year_strings
            
        thredds_dir = thredds_root_dir + "/" + scenario


        for v in range(len(variables)):
            
            print("# =========================================")

            variable       =       variables[v]
            version_string = version_strings[v]
            
            fnl_thredds_file = variable + "___" + model + "___" + scenario + ".nc"
            
            globus_dir  = globus_root_dir + "/" + scenario + "/" + variable 
                        
            for year_string in year_strings:
                
                print("# -----------------------------------------")
            
                globus_file      = variable + "." + model + "." + scenario + "." + year_string + "." + version_string + ".nc"
                
                globus_file_loc  = globus_dir + "/" + globus_file

                if (scenario == "historical"):
                    mid_thredds_file = variable + "___" + model + "___" + scenario + ".nc"
                else:
                    mid_thredds_file = variable + "___" + model + "___" + scenario + "___" + year_string + ".nc"
                    
                mid_thredds_loc = thredds_dir + "/" + mid_thredds_file

            
                print('nohup ncatted -h -O -a           ,global,d,,                 ' + globus_file_loc)
                
                print(" ")
                
                print('nohup ncatted -h -O -a              ,lat,d,,                 ' + globus_file_loc)
                print('nohup ncatted -h -O -a   description,lat,c,c,"Latitude"      ' + globus_file_loc)
                print('nohup ncatted -h -O -a     long_name,lat,c,c,"Latitude"      ' + globus_file_loc)
                print('nohup ncatted -h -O -a standard_name,lat,c,c,"latitude"      ' + globus_file_loc)
                print('nohup ncatted -h -O -a         units,lat,c,c,"degrees_north" ' + globus_file_loc)
                print('nohup ncatted -h -O -a          axis,lat,c,c,"Y"             ' + globus_file_loc)
                
                print(" ")
                
                print('nohup ncatted -h -O -a              ,lon,d,,                ' + globus_file_loc)
                print('nohup ncatted -h -O -a   description,lon,c,c,"Longitude"    ' + globus_file_loc)
                print('nohup ncatted -h -O -a     long_name,lon,c,c,"Longitude"    ' + globus_file_loc)
                print('nohup ncatted -h -O -a standard_name,lon,c,c,"longitude"    ' + globus_file_loc)
                print('nohup ncatted -h -O -a         units,lon,c,c,"degrees_east" ' + globus_file_loc)
                print('nohup ncatted -h -O -a          axis,lon,c,c,"X"            ' + globus_file_loc)

                print(" ")

                if (variable == "tasmax"):
                    
                    print('nohup ' + "ncap2 -h -d 8 --script 'where(tasmax > 1e9) tasmax=(-32493.85)'  " + globus_file_loc + ' ' + globus_file_loc + '.temp.nc' )
                    print('nohup mv -fv ' + globus_file_loc + '.temp.nc' + ' ' +  globus_file_loc   )

                    print(" ")

                    print('nohup ' + "ncap2 -h -d 8 --script 'tasmax=short(round((tasmax-273.15)*10))' " + globus_file_loc + ' ' + globus_file_loc + '.temp.nc' )
                    print('nohup mv -fv ' + globus_file_loc + '.temp.nc' + ' ' +  globus_file_loc   )

                    print(" ")

                    print('nohup ncatted -h -O -a              ,tasmax,d,,                                 ' + globus_file_loc)
                    print('nohup ncatted -h -O -a   description,tasmax,c,c,"2-m Max Daily Air Temperature" ' + globus_file_loc)
                    print('nohup ncatted -h -O -a     long_name,tasmax,c,c,"2-m Max Daily Air Temperature" ' + globus_file_loc)
                    print('nohup ncatted -h -O -a standard_name,tasmax,c,c,"air_temperature"               ' + globus_file_loc)
                    print('nohup ncatted -h -O -a         units,tasmax,c,c,"degC"                          ' + globus_file_loc)
                    print('nohup ncatted -h -O -a  cell_methods,tasmax,c,c,"time: maximum"                 ' + globus_file_loc)
                    print('nohup ncatted -h -O -a    _FillValue,tasmax,c,s,-32767                          ' + globus_file_loc)
                    print('nohup ncatted -h -O -a    add_offset,tasmax,c,f,0                               ' + globus_file_loc)
                    print('nohup ncatted -h -O -a  scale_factor,tasmax,c,f,0.1                             ' + globus_file_loc)
                    
                    print(" ")
                   
                elif (variable == "tasmin"):
                    
                    print('nohup ' + "ncap2 -h -d 8 --script 'where(tasmin > 1e9) tasmin=(-3003.55)'  " + globus_file_loc + ' ' + globus_file_loc + '.temp.nc' )
                    print('nohup mv -fv ' + globus_file_loc + '.temp.nc' + ' ' +  globus_file_loc   )

                    print(" ")

                    print("nohup ncap2 -h -d 8 --script 'tasmin=short(round((tasmin-273.15)*10))' " + globus_file_loc + ' ' + globus_file_loc + '.temp.nc' )
                    print('nohup mv -fv ' + globus_file_loc + '.temp.nc' + ' ' +  globus_file_loc   )

                    print(" ")

                    print('nohup ncatted -h -O -a              ,tasmin,d,,                                 ' + globus_file_loc)
                    print('nohup ncatted -h -O -a   description,tasmin,c,c,"2-m Min Daily Air Temperature" ' + globus_file_loc)
                    print('nohup ncatted -h -O -a     long_name,tasmin,c,c,"2-m Min Daily Air Temperature" ' + globus_file_loc)
                    print('nohup ncatted -h -O -a standard_name,tasmin,c,c,"air_temperature"               ' + globus_file_loc)
                    print('nohup ncatted -h -O -a         units,tasmin,c,c,"degC"                          ' + globus_file_loc)
                    print('nohup ncatted -h -O -a  cell_methods,tasmin,c,c,"time: minimum"                 ' + globus_file_loc)
                    print('nohup ncatted -h -O -a    _FillValue,tasmin,c,s,-32767                          ' + globus_file_loc)
                    print('nohup ncatted -h -O -a    add_offset,tasmin,c,f,0                               ' + globus_file_loc)
                    print('nohup ncatted -h -O -a  scale_factor,tasmin,c,f,0.1                             ' + globus_file_loc)

                    print(" ")

                else: 
                
                    print("nohup ncap2 -h -d 8 --script 'where(pr > 1e9) pr=(-32767./864000.)' " + globus_file_loc + ' ' + globus_file_loc + '.temp.nc' )
                    print('nohup mv -fv ' + globus_file_loc + '.temp.nc' + ' ' +  globus_file_loc   )

                    print(" ")

                    print('nohup ' + "ncap2 -h -d 8 --script 'pr=short(round(pr*864000.))' " + globus_file_loc + ' ' + globus_file_loc + '.temp.nc' )
                    print('nohup mv -fv ' + globus_file_loc + '.temp.nc' + ' ' +  globus_file_loc   )

                    print(" ")

                    print('nohup ncatted -h -O -a              ,pr,d,,                             ' + globus_file_loc)
                    print('nohup ncatted -h -O -a   description,pr,c,c,"Daily Total Precipitation" ' + globus_file_loc)
                    print('nohup ncatted -h -O -a     long_name,pr,c,c,"Daily Total Precipitation" ' + globus_file_loc)
                    print('nohup ncatted -h -O -a standard_name,pr,c,c,"precipitation_amount"      ' + globus_file_loc)
                    print('nohup ncatted -h -O -a         units,pr,c,c,"kg m-2"                    ' + globus_file_loc)
                    print('nohup ncatted -h -O -a  cell_methods,pr,c,c,"time: sum"                 ' + globus_file_loc)
                    print('nohup ncatted -h -O -a    _FillValue,pr,c,s,-32767                      ' + globus_file_loc)
                    print('nohup ncatted -h -O -a    add_offset,pr,c,f,0                           ' + globus_file_loc)
                    print('nohup ncatted -h -O -a  scale_factor,pr,c,f,0.1                         ' + globus_file_loc)

                    print(" ")

                print('nohup mv -fv ' + globus_file_loc + ' ' +  mid_thredds_loc   )

                print(" ")


            
            
            

# XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
# =========================================
# -----------------------------------------
nohup ncatted -h -O -a           ,global,d,,                 /data/GLOBUS/wjc/historical/tasmax/tasmax.GFDL-ESM4.historical.1950-2014.LOCA_16thdeg_v20220413.nc
 
nohup ncatted -h -O -a              ,lat,d,,                 /data/GLOBUS/wjc/historical/tasmax/tasmax.GFDL-ESM4.historical.1950-2014.LOCA_16thdeg_v20220413.nc
nohup ncatted -h -O -a   description,lat,c,c,"Latitude"      /data/GLOBUS/wjc/historical/tasmax/tasmax.GFDL-ESM4.historical.1950-2014.LOCA_16thdeg_v20220413.nc
nohup ncatted -h -O -a     long_name,lat,c,c,"Latitude"      /data/GLOBUS/wjc/historical/tasmax/tasmax.GFDL-ESM4.historical.1950-2014.LOCA_16thdeg_v20220413.nc
nohup ncatted -h -O -a standard_name,lat,c,c,"latitude"      /data/GLOBUS/wjc/historical/tasmax/tasmax.GFDL-ESM4.historical.1950-2014.LOCA_16thdeg_v20220413.nc
nohup ncatted -h -O -a         units,lat,c,c,"degrees_north" /data

In [5]:
tasmin=(-3003.55)
tasmin

-3003.55

In [6]:
np.short(round((tasmin-273.15)*10))

-32767

In [7]:
pr=(-32767./864000.)
pr

-0.03792476851851852

In [8]:
np.short(round(pr*864000.))

-32767

 File Names:

```
./historical/tasmin/tasmin.GFDL-ESM4.historical.r1i1p1f1.1950-2014.LOCA_16thdeg_v20220413.nc
./historical/tasmax/tasmax.GFDL-ESM4.historical.r1i1p1f1.1950-2014.LOCA_16thdeg_v20220413.nc
./historical/pr/pr.GFDL-ESM4.historical.r1i1p1f1.1950-2014.LOCA_16thdeg_v20220519.nc



./ssp245/tasmin/tasmin.GFDL-ESM4.ssp245.r1i1p1f1.2075-2100.LOCA_16thdeg_v20220413.nc  
./ssp245/tasmin/tasmin.GFDL-ESM4.ssp245.r1i1p1f1.2015-2044.LOCA_16thdeg_v20220413.nc
./ssp245/tasmin/tasmin.GFDL-ESM4.ssp245.r1i1p1f1.2045-2074.LOCA_16thdeg_v20220413.nc

./ssp245/tasmax/tasmax.GFDL-ESM4.ssp245.r1i1p1f1.2075-2100.LOCA_16thdeg_v20220413.nc  
./ssp245/tasmax/tasmax.GFDL-ESM4.ssp245.r1i1p1f1.2015-2044.LOCA_16thdeg_v20220413.nc
./ssp245/tasmax/tasmax.GFDL-ESM4.ssp245.r1i1p1f1.2045-2074.LOCA_16thdeg_v20220413.nc

./ssp245/pr/pr.GFDL-ESM4.ssp245.r1i1p1f1.2075-2100.LOCA_16thdeg_v20220519.nc  
./ssp245/pr/pr.GFDL-ESM4.ssp245.r1i1p1f1.2015-2044.LOCA_16thdeg_v20220519.nc
./ssp245/pr/pr.GFDL-ESM4.ssp245.r1i1p1f1.2045-2074.LOCA_16thdeg_v20220519.nc



./ssp370/tasmin/tasmin.GFDL-ESM4.ssp370.r1i1p1f1.2075-2100.LOCA_16thdeg_v20220413.nc  
./ssp370/tasmin/tasmin.GFDL-ESM4.ssp370.r1i1p1f1.2015-2044.LOCA_16thdeg_v20220413.nc
./ssp370/tasmin/tasmin.GFDL-ESM4.ssp370.r1i1p1f1.2045-2074.LOCA_16thdeg_v20220413.nc

./ssp370/tasmax/tasmax.GFDL-ESM4.ssp370.r1i1p1f1.2075-2100.LOCA_16thdeg_v20220413.nc  
./ssp370/tasmax/tasmax.GFDL-ESM4.ssp370.r1i1p1f1.2015-2044.LOCA_16thdeg_v20220413.nc
./ssp370/tasmax/tasmax.GFDL-ESM4.ssp370.r1i1p1f1.2045-2074.LOCA_16thdeg_v20220413.nc

./ssp370/pr/pr.GFDL-ESM4.ssp370.r1i1p1f1.2075-2100.LOCA_16thdeg_v20220519.nc  
./ssp370/pr/pr.GFDL-ESM4.ssp370.r1i1p1f1.2015-2044.LOCA_16thdeg_v20220519.nc
./ssp370/pr/pr.GFDL-ESM4.ssp370.r1i1p1f1.2045-2074.LOCA_16thdeg_v20220519.nc



./ssp585/tasmin/tasmin.GFDL-ESM4.ssp585.r1i1p1f1.2075-2100.LOCA_16thdeg_v20220413.nc  
./ssp585/tasmin/tasmin.GFDL-ESM4.ssp585.r1i1p1f1.2015-2044.LOCA_16thdeg_v20220413.nc
./ssp585/tasmin/tasmin.GFDL-ESM4.ssp585.r1i1p1f1.2045-2074.LOCA_16thdeg_v20220413.nc

./ssp585/tasmax/tasmax.GFDL-ESM4.ssp585.r1i1p1f1.2075-2100.LOCA_16thdeg_v20220413.nc  
./ssp585/tasmax/tasmax.GFDL-ESM4.ssp585.r1i1p1f1.2015-2044.LOCA_16thdeg_v20220413.nc
./ssp585/tasmax/tasmax.GFDL-ESM4.ssp585.r1i1p1f1.2045-2074.LOCA_16thdeg_v20220413.nc

./ssp585/pr/pr.GFDL-ESM4.ssp585.r1i1p1f1.2075-2100.LOCA_16thdeg_v20220519.nc  
./ssp585/pr/pr.GFDL-ESM4.ssp585.r1i1p1f1.2015-2044.LOCA_16thdeg_v20220519.nc
./ssp585/pr/pr.GFDL-ESM4.ssp585.r1i1p1f1.2045-2074.LOCA_16thdeg_v20220519.nc
```


In [9]:

for rcp in rcps:
    for ensemble in range(len(ensembles)):



        original_file_nc_name = "rel_humid." + \
                                ensemble    + \
                                "."         + \
                                rcp         + \
                                "."         + \
                                date        + \
                                "."         + \
                                variable    + \
                                "."         + \
                                suffix

        thredds_url_full = ftp_prefix + rcp + "/" + variable + "/"

        output_file_nc_name =   "NGP_LOCA"    + \
                                "_"           + \
                                variable2      +  \
                                "_"           + \
                                ensemble2    + \
                                "_"         + \
                                rcp

        print("==============================")
        print(output_file_nc_name)
        print(thredds_root  + rcp + "/" + variable2 )
        os.chdir(thredds_root  + rcp + "/" + variable2 )

        os.system('wget --no-verbose ' + thredds_url_full + original_file_nc_name + '.nc.gz')
        os.system('gunzip ./' + original_file_nc_name + '.nc.gz')

        print(" ")
        os.system('ncdump -h ./' + original_file_nc_name + '.nc')
        print(" ")

        print("create spatial subset")
        os.system('ncea -h -4 -d lat,141,442 -d lon,165,614 ' + rcptimedims + original_file_nc_name + '.nc     ' + output_file_nc_name + '_clip.nc')
        print(" ")

        print("Strip Down Attributes")           
        os.system('ncatted -h -O -a   description,,d,, ' + output_file_nc_name + '_clip.nc ')
        os.system('ncatted -h -O -a     long_name,,d,, ' + output_file_nc_name + '_clip.nc ')
        os.system('ncatted -h -O -a standard_name,,d,, ' + output_file_nc_name + '_clip.nc ')
        os.system('ncatted -h -O -a         units,,d,, ' + output_file_nc_name + '_clip.nc ')
        os.system('ncatted -h -O -a          axis,,d,, ' + output_file_nc_name + '_clip.nc ')
        os.system('ncatted -h -O -a        bounds,,d,, ' + output_file_nc_name + '_clip.nc ')
        os.system('ncatted -h -O -a  cell_methods,,d,, ' + output_file_nc_name + '_clip.nc ')
        print(" ")

        print("Time Attributes")
        os.system('ncatted -h -O -a   description,time,c,c,"Time"      ' + output_file_nc_name + '_clip.nc ')
        os.system('ncatted -h -O -a     long_name,time,c,c,"Time"      ' + output_file_nc_name + '_clip.nc ')
        os.system('ncatted -h -O -a standard_name,time,c,c,"time"      ' + output_file_nc_name + '_clip.nc ')
        os.system('ncatted -h -O -a         units,time,c,c,"days since 1900-01-01 00:00:00" '+ output_file_nc_name +'_clip.nc ')
        os.system('ncatted -h -O -a          axis,time,c,c,"T"         ' + output_file_nc_name + '_clip.nc ')
        os.system('ncatted -h -O -a        bounds,time,c,c,"time_bnds" ' + output_file_nc_name + '_clip.nc ')
        print(" ")

        print("Latitude Attributes")           
        os.system('ncatted -h -O -a   description,lat,c,c,"Latitude"      ' + output_file_nc_name + '_clip.nc ')
        os.system('ncatted -h -O -a     long_name,lat,c,c,"Latitude"      ' + output_file_nc_name + '_clip.nc ')
        os.system('ncatted -h -O -a standard_name,lat,c,c,"latitude"      ' + output_file_nc_name + '_clip.nc ')
        os.system('ncatted -h -O -a         units,lat,c,c,"degrees_north" ' + output_file_nc_name + '_clip.nc ')
        os.system('ncatted -h -O -a          axis,lat,c,c,"Y"             ' + output_file_nc_name + '_clip.nc ')
        os.system('ncatted -h -O -a        bounds,lat,c,c,"lat_bnds"      ' + output_file_nc_name + '_clip.nc ')
        print(" ")

        print("Longitude Attributes")              
        os.system('ncatted -h -O -a   description,lon,c,c,"Longitude"    ' + output_file_nc_name + '_clip.nc ')
        os.system('ncatted -h -O -a     long_name,lon,c,c,"Longitude"    ' + output_file_nc_name + '_clip.nc ')
        os.system('ncatted -h -O -a standard_name,lon,c,c,"longitude"    ' + output_file_nc_name + '_clip.nc ')
        os.system('ncatted -h -O -a         units,lon,c,c,"degrees_east" ' + output_file_nc_name + '_clip.nc ')
        os.system('ncatted -h -O -a          axis,lon,c,c,"X"            ' + output_file_nc_name + '_clip.nc ')
        os.system('ncatted -h -O -a        bounds,lon,c,c,"lon_bnds"     ' + output_file_nc_name + '_clip.nc ')
        print(" ")

        print(variable2, ensemble, rcp," Attributes")              
        os.system('ncatted -h -O -a description,'  + variable + ',c,c,' + '"' + lname + '"       ' + output_file_nc_name + '_clip.nc ')
        os.system('ncatted -h -O -a long_name,'    + variable + ',c,c,' + '"' + lname + '"       ' + output_file_nc_name + '_clip.nc ')
        os.system('ncatted -h -O -a standard_name,'+ variable + ',c,c,' + '"relative_humidity"   ' + output_file_nc_name + '_clip.nc ')
        os.system('ncatted -h -O -a units,'        + variable + ',c,c,' + '"percent"             ' + output_file_nc_name + '_clip.nc ')
        os.system('ncatted -h -O -a cell_methods,' + variable + ',c,c,' + '"time: ' + cmeth + '" ' + output_file_nc_name + '_clip.nc ')
        print(" ")

        print("Compress ",variable2, ensemble, rcp)              
        os.system("ncap2 -h --deflate 8 -s '" + variable + "=short(round(" + variable + "*10))' " + output_file_nc_name + '_clip.nc ' + output_file_nc_name + '_short.nc')
        os.system('rm ' + output_file_nc_name +'_clip.nc')
        os.system('ncrename -h -v ' + variable + ',' + variable2+ '  ' + output_file_nc_name + '_short.nc')
        os.system('ncatted -h -O -a   add_offset,' + variable2 + ',c,f,0   ' + output_file_nc_name + '_short.nc')
        os.system('ncatted -h -O -a scale_factor,' + variable2 + ',c,f,0.1 ' + output_file_nc_name + '_short.nc')
        os.system('ncatted -h -O -a      missing,' + variable2 + ',c,s,-1 ' + output_file_nc_name + '_short.nc')
        print(" ")



        print("Compress ",variable2, ensemble, rcp)              
        os.system("ncap2 -h --deflate 8 -s '" + bounds_script_command + "' " + output_file_nc_name + '_short.nc ' + output_file_nc_name + '.nc')
        os.system('rm ' + output_file_nc_name + '_short.nc')
        print(" ")

        print("Time Bounds Attributes")              
        os.system('ncatted -h -O -a   description,time_bnds,c,c,"Time Bounds" '                    + output_file_nc_name + '.nc')
        os.system('ncatted -h -O -a     long_name,time_bnds,c,c,"Time Bounds" '                    + output_file_nc_name + '.nc')
        os.system('ncatted -h -O -a standard_name,time_bnds,c,c,"time" '                           + output_file_nc_name + '.nc')
        os.system('ncatted -h -O -a         units,time_bnds,c,c,"days since 1900-01-01 00:00:00" ' + output_file_nc_name + '.nc')
        print(" ")

        print("Latitude Bounds Attributes")
        os.system('ncatted -h -O -a   description,lat_bnds,c,c,"Latitude Bounds" '  + output_file_nc_name + '.nc')
        os.system('ncatted -h -O -a     long_name,lat_bnds,c,c,"Latitude Bounds" '  + output_file_nc_name + '.nc')
        os.system('ncatted -h -O -a standard_name,lat_bnds,c,c,"latitude" '         + output_file_nc_name + '.nc')
        os.system('ncatted -h -O -a         units,lat_bnds,c,c,"degrees_north" '    + output_file_nc_name + '.nc')
        print(" ")

        print("Longitude Bounds Attributes")                
        os.system('ncatted -h -O -a   description,lon_bnds,c,c,"Longitude Bounds" ' + output_file_nc_name + '.nc')
        os.system('ncatted -h -O -a     long_name,lon_bnds,c,c,"Longitude Bounds" ' + output_file_nc_name + '.nc')
        os.system('ncatted -h -O -a standard_name,lon_bnds,c,c,"longitude" '        + output_file_nc_name + '.nc')
        os.system('ncatted -h -O -a         units,lon_bnds,c,c,"degrees_east" '     + output_file_nc_name + '.nc')
        os.system('rm ' + original_file_nc_name + '.nc')






NameError: name 'rcps' is not defined